In [1]:
from __future__ import annotations

from pathlib import Path

import geopandas as gpd

from slope_area.builder import (
    BuilderConfig,
    DEMSource,
    OutletPlotBuilder,
    ResolutionPlotBuilder,
    Trial,
    TrialConfig,
    StaticVRT,
)
from slope_area.paths import PROJ_ROOT
from slope_area.features import DEMTilesBuilder, GeneralizedDEM, Outlets, VRT, DEMTiles
from slope_area.geomorphometry import HydrologicAnalysisConfig
from slope_area.logger import create_logger

logger = create_logger(__name__)

02:28:14 DEBUG slopeArea.slope_area.config          is_notebook(): Code is running in notebook


In [2]:
# ---- Paths ----
dem_dir = Path(r'Z:\\')
dem_tiles = PROJ_ROOT / 'data' / 'raw' / 'dem_tiles.fgb'
generalized_dem = PROJ_ROOT / 'data' / 'raw' / 'dem_30m.tif'
generalized_dem_out = (
    PROJ_ROOT / 'data' / 'processed' / generalized_dem.stem
)
outlets = PROJ_ROOT / 'data' / 'raw' / 'outlets.shp'
dem_dir_epsg = 3844
out_dir = PROJ_ROOT / 'data' / 'processed'

# ---- Run configs ----
plot_kind = 'resolution'
outlet_name = 'gully 2'
resolutions = [(res, res) for res in range(5, 15)]

# ---- Init objects ----
tiles = DEMTilesBuilder(
    dem_dir, dem_dir_epsg=dem_dir_epsg, tiles=dem_tiles
).build()
generalized_dem = GeneralizedDEM(
    path=generalized_dem, out_dir=generalized_dem_out
)
dem_source = DEMSource(dem_dir, tiles, generalized_dem)
hydrologic_analysis_config = HydrologicAnalysisConfig(
    streams_flow_accumulation_threshold=1000, outlet_snap_distance=100
)

logger.info('Reading outlets at %s' % outlets)
gdf = gpd.read_file(outlets).sort_values(by='name')
gdf = gdf[gdf['is_gully'] == 1]
outlets = Outlets.from_gdf(gdf, name_field='name')

02:28:17 INFO slopeArea.slope_area.features.DEMTilesBuilder                build(): Found DEM tiles at C:\Users\alex\Desktop\slope-area\data\raw\dem_tiles.fgb
02:28:17 INFO slopeArea.__main__             <module>(): Reading outlets at C:\Users\alex\Desktop\slope-area\data\raw\outlets.shp


In [3]:
builder_config = BuilderConfig(
    hydrologic_analysis_config, out_dir / outlet_name, max_workers=3
)
outlet = [outlet for outlet in outlets if outlet.name == outlet_name]
ResolutionPlotBuilder(
    builder_config, dem_source, outlet[0], resolutions
).build()

02:28:17 INFO slopeArea.slope_area.features.DEMTiles          get_wbw_env(): Initialized Whitebox Environment
02:28:17 INFO slopeArea.slope_area.features.DEMTiles _from_outlet_watershed(): Infering DEM tiles based on the outlet
02:28:17 INFO slopeArea.slope_area.config          get_wbw_env(): Initialized Whitebox Environment
02:28:17 INFO slopeArea.slope_area.features      get_flow_output(): Found rasters dem_30m_dem_preproc.tif, dem_30m_d8_pointer.tif, dem_30m_flow_accumulation.tif
02:28:43 INFO slopeArea.slope_area.features.DEMTiles       write_whitebox(): Saving Vector object to C:\Users\alex\Desktop\slope-area\data\processed\gully 2\watershed.shp
02:28:45 INFO slopeArea.slope_area.features.DEMTiles         from_polygon(): Extracted 6 tiles based on the polygon <POLYGON ((650962 642080, 651052 642080, 651052 642050, 651172 642050, 65117...>
02:28:45 INFO slopeArea.slope_area.features              wrapper(): DEMTiles.from_outlet executed in 27.811711 seconds
02:28:45 INFO slopeArea.s

Output()

02:30:12 INFO slopeArea.slope_area.plot      slope_area_grid(): Creating slope area plot with config SlopeAreaPlotConfig(col_wrap=-1, height=5, aspect=1)
02:30:12 INFO slopeArea.slope_area.plot         get_col_wrap(): Infered 5 cols for the plot
02:30:23 INFO slopeArea.slope_area.plot      slope_area_grid(): Saved slope area plot at C:\Users\alex\Desktop\slope-area\data\processed\gully 2\slope_area.png


In [4]:
builder_config = BuilderConfig(
    hydrologic_analysis_config, out_dir, max_workers=2
)
OutletPlotBuilder(
    builder_config, dem_source, outlets=outlets, resolution=(20, 20)
).build()

Output()

02:42:33 INFO slopeArea.slope_area.plot      slope_area_grid(): Creating slope area plot with config SlopeAreaPlotConfig(col_wrap=-1, height=5, aspect=1)
02:42:33 INFO slopeArea.slope_area.plot         get_col_wrap(): Infered 4 cols for the plot
02:43:06 INFO slopeArea.slope_area.plot      slope_area_grid(): Saved slope area plot at C:\Users\alex\Desktop\slope-area\data\processed\slope_area.png


In [5]:
outlet = outlets[0]
out_dir = out_dir / outlet.name
trial = Trial(
    TrialConfig(
        name=outlet.name,
        outlet=outlets[0],
        resolution=(10, 10),
        hydrologic_analysis_config=hydrologic_analysis_config,
        dem_provider=StaticVRT(
            VRT.from_dem_tiles(
                DEMTiles.from_outlet(
                    dem_source,
                    outlet,
                    out_dir,
                    hydrologic_analysis_config.outlet_snap_distance,
                ),
                out_dir / 'dem.vrt',
            )
        ),
        out_dir=out_dir,
    )
).run()

02:43:06 INFO slopeArea.slope_area.features.DEMTiles _from_outlet_watershed(): Infering DEM tiles based on the outlet
02:43:10 INFO slopeArea.slope_area.features.DEMTiles       write_whitebox(): Saving Vector object to C:\Users\alex\Desktop\slope-area\data\processed\gully 1\watershed.shp
02:43:12 INFO slopeArea.slope_area.features.DEMTiles         from_polygon(): Extracted 4 tiles based on the polygon <MULTIPOLYGON (((717832 495800, 717922 495800, 717922 495770, 717862 495770,...>
02:43:12 INFO slopeArea.slope_area.features              wrapper(): DEMTiles.from_outlet executed in 6.737821 seconds
02:43:12 INFO slopeArea.slope_area.features       from_dem_tiles(): Building VRT from 4 rasters
02:43:14 INFO slopeArea.slope_area.builder.Trial              execute(): 
02:43:14 INFO slopeArea.slope_area.builder.Trial              execute(): Getting the DEM raster
02:43:14 INFO slopeArea.slope_area.builder.Trial          get_wbw_env(): Initialized Whitebox Environment


02:43:14 WARNING slopeArea.slope_area.builder.Trial    get_resampled_dem(): CRS of dem.vrt is unprojected. Defaulting to the outlet CRS EPSG:3844


02:43:14 INFO slopeArea.slope_area.builder.Trial             resample(): Resampling Raster dem.vrt at resolution=(10, 10)
02:43:16 INFO slopeArea.slope_area.builder.Trial       preprocess_dem(): Breaching depressions in the DEM
02:43:16 INFO slopeArea.slope_area.builder.Trial             read_dem(): Reading DEM dem_resampled.tif
02:43:16 DEBUG slopeArea.slope_area.geomorphometry              wrapper(): HydrologicAnalysis.read_dem executed in 0.005413 seconds
02:43:16 INFO slopeArea.slope_area.builder.Trial       preprocess_dem(): Breaching single-cell pits
02:43:16 DEBUG slopeArea.slope_area.geomorphometry              wrapper(): HydrologicAnalysis.preprocess_dem executed in 0.015931 seconds
02:43:16 INFO slopeArea.slope_area.builder.Trial         compute_flow(): Computing the D8 pointer
02:43:16 INFO slopeArea.slope_area.builder.Trial         compute_flow(): Computing the flow accumulation
02:43:16 DEBUG slopeArea.slope_area.geomorphometry              wrapper(): HydrologicAnalysis.co